In [2]:
import pandas as pd
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from  tensorflow.keras.utils import to_categorical
from keras.models import Sequential
import tensorflow as tf

In [3]:
dataframe = pd.read_csv('data/chinese_mnist.csv')
index = dataframe.iloc[:,:-2].values 
value = dataframe["code"].values
chineseNumber = dataframe["character"]

In [4]:
print(index.shape)
print(len(value))

(15000, 3)
15000


In [ ]:
filename_label_link = {}

filename_list = []
label_list = []
X = []
y = [] 
for i in range(0,len(value)):
    x = index[i]
    filename = "input_%s_%s_%s.jpg" % (x[0], x[1], x[2])
    val = value[i] 
    filename_label_link[filename] = val
    filename_list.append(filename)
    label_list.append(val)
    a = cv2.imread("data/data/%s"%filename)
    X.append(a)
    y.append(val)

In [6]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train[0])
print(y_test[0])

(12000,)
(3000,)
(12000, 16)
(3000, 16)
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [7]:
model = Sequential()
model.add(tf.keras.layers.Conv2D(128, 3,
                                 padding = "same", 
                                 activation = "relu", 
                                 input_shape=[64, 64, 3]))


model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(128, 3,
                                 padding = "same", 
                                 activation = "relu", 
                                 input_shape=[64, 64, 3]))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides = 2))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128,activation = "relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(y_train.shape[1],activation = "softmax"))
print('summary total')
model.summary()
model.compile(optimizer='adam',
              loss = "categorical_crossentropy", 
              metrics = ['accuracy'])  


summary total
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 32, 32, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 128)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 16, 16

In [8]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

In [ ]:
# 绘制训练和验证损失曲线
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 绘制训练和验证准确率曲线
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()